# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 10 2023 8:30AM,259053,10,MTR-23-0294,"Senseonics, Incorporated",Released
1,Mar 9 2023 4:11PM,259056,16,SSF_C2W11,Sartorius Stedim Filters Inc.,Released
2,Mar 9 2023 4:03PM,259054,19,60021972,"GCH Granules USA, Inc.",Released
3,Mar 9 2023 4:03PM,259054,19,60021973,"GCH Granules USA, Inc.",Released
4,Mar 9 2023 4:03PM,259054,19,60021974,"GCH Granules USA, Inc.",Released
5,Mar 9 2023 4:03PM,259054,19,60021975,"GCH Granules USA, Inc.",Released
6,Mar 9 2023 4:03PM,259054,19,60021976,"GCH Granules USA, Inc.",Released
7,Mar 9 2023 4:03PM,259054,19,60021977,"GCH Granules USA, Inc.",Released
8,Mar 9 2023 4:03PM,259054,19,60021996,"GCH Granules USA, Inc.",Released
9,Mar 9 2023 4:03PM,259054,19,60021997,"GCH Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,259047,Released,1
17,259052,Released,1
18,259053,Released,1
19,259054,Released,27
20,259056,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259047,NaN,NaN,1.0
259052,NaN,NaN,1.0
259053,NaN,NaN,1.0
259054,NaN,NaN,27.0
259056,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258979,0.0,1.0,0.0
258980,53.0,0.0,4.0
258993,0.0,0.0,1.0
258997,25.0,17.0,1.0
259011,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258979,0,1,0
258980,53,0,4
258993,0,0,1
258997,25,17,1
259011,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258979,0,1,0
1,258980,53,0,4
2,258993,0,0,1
3,258997,25,17,1
4,259011,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258979,,1,
1,258980,53,,4
2,258993,,,1
3,258997,25,17,1
4,259011,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated"
1,Mar 9 2023 4:11PM,259056,16,Sartorius Stedim Filters Inc.
2,Mar 9 2023 4:03PM,259054,19,"GCH Granules USA, Inc."
29,Mar 9 2023 3:33PM,259052,21,"NBTY Global, Inc."
30,Mar 9 2023 3:23PM,259047,16,Sartorius Bioprocess Solutions
31,Mar 9 2023 3:22PM,259046,16,Sartorius Stedim Filters Inc.
32,Mar 9 2023 2:50PM,259042,19,Eli Lilly and Company
33,Mar 9 2023 1:24PM,259032,19,"AdvaGen Pharma, Ltd"
34,Mar 9 2023 12:47PM,258997,15,"Alliance Pharma, Inc."
77,Mar 9 2023 12:25PM,259020,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated",,,1
1,Mar 9 2023 4:11PM,259056,16,Sartorius Stedim Filters Inc.,,,1
2,Mar 9 2023 4:03PM,259054,19,"GCH Granules USA, Inc.",,,27
3,Mar 9 2023 3:33PM,259052,21,"NBTY Global, Inc.",,,1
4,Mar 9 2023 3:23PM,259047,16,Sartorius Bioprocess Solutions,,,1
5,Mar 9 2023 3:22PM,259046,16,Sartorius Stedim Filters Inc.,,,1
6,Mar 9 2023 2:50PM,259042,19,Eli Lilly and Company,,,1
7,Mar 9 2023 1:24PM,259032,19,"AdvaGen Pharma, Ltd",,1,
8,Mar 9 2023 12:47PM,258997,15,"Alliance Pharma, Inc.",25,17,1
9,Mar 9 2023 12:25PM,259020,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated",1,,
1,Mar 9 2023 4:11PM,259056,16,Sartorius Stedim Filters Inc.,1,,
2,Mar 9 2023 4:03PM,259054,19,"GCH Granules USA, Inc.",27,,
3,Mar 9 2023 3:33PM,259052,21,"NBTY Global, Inc.",1,,
4,Mar 9 2023 3:23PM,259047,16,Sartorius Bioprocess Solutions,1,,
5,Mar 9 2023 3:22PM,259046,16,Sartorius Stedim Filters Inc.,1,,
6,Mar 9 2023 2:50PM,259042,19,Eli Lilly and Company,1,,
7,Mar 9 2023 1:24PM,259032,19,"AdvaGen Pharma, Ltd",,1,
8,Mar 9 2023 12:47PM,258997,15,"Alliance Pharma, Inc.",1,17,25
9,Mar 9 2023 12:25PM,259020,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated",1,,
1,Mar 9 2023 4:11PM,259056,16,Sartorius Stedim Filters Inc.,1,,
2,Mar 9 2023 4:03PM,259054,19,"GCH Granules USA, Inc.",27,,
3,Mar 9 2023 3:33PM,259052,21,"NBTY Global, Inc.",1,,
4,Mar 9 2023 3:23PM,259047,16,Sartorius Bioprocess Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated",1.0,NaN,NaN
1,Mar 9 2023 4:11PM,259056,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
2,Mar 9 2023 4:03PM,259054,19,"GCH Granules USA, Inc.",27.0,NaN,NaN
3,Mar 9 2023 3:33PM,259052,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Mar 9 2023 3:23PM,259047,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 8:30AM,259053,10,"Senseonics, Incorporated",1.0,0.0,0.0
1,Mar 9 2023 4:11PM,259056,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
2,Mar 9 2023 4:03PM,259054,19,"GCH Granules USA, Inc.",27.0,0.0,0.0
3,Mar 9 2023 3:33PM,259052,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Mar 9 2023 3:23PM,259047,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1036056,3.0,1.0,0.0
12,518037,1.0,1.0,0.0
15,258997,1.0,17.0,25.0
16,777149,3.0,0.0,0.0
19,1295122,33.0,1.0,53.0
21,518072,2.0,0.0,0.0
22,258993,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1036056,3.0,1.0,0.0
1,12,518037,1.0,1.0,0.0
2,15,258997,1.0,17.0,25.0
3,16,777149,3.0,0.0,0.0
4,19,1295122,33.0,1.0,53.0
5,21,518072,2.0,0.0,0.0
6,22,258993,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,3.0,1.0,0.0
1,12,1.0,1.0,0.0
2,15,1.0,17.0,25.0
3,16,3.0,0.0,0.0
4,19,33.0,1.0,53.0
5,21,2.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,3.0
1,12,Released,1.0
2,15,Released,1.0
3,16,Released,3.0
4,19,Released,33.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,0.0,0.0,25.0,0.0,53.0,0.0,0.0
Executing,1.0,1.0,17.0,0.0,1.0,0.0,0.0
Released,3.0,1.0,1.0,3.0,33.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,25.0,0.0,53.0,0.0,0.0
1,Executing,1.0,1.0,17.0,0.0,1.0,0.0,0.0
2,Released,3.0,1.0,1.0,3.0,33.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,25.0,0.0,53.0,0.0,0.0
1,Executing,1.0,1.0,17.0,0.0,1.0,0.0,0.0
2,Released,3.0,1.0,1.0,3.0,33.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()